In [1]:
import pandas as pd
import sqlite3
import os
import time
import matplotlib.pyplot as plt
import seaborn as sns

In [58]:
# # Set display options to show all rows and columns
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
save_path = f"{os.getcwd()}/visualizations/"
save_path

'/home/jovyan/work/notebooks/visualizations/'

In [4]:
def normalize_column_names(df):
    """
    Normalize the column names of a Pandas DataFrame.
    
    Parameters:
    df (pd.DataFrame): The DataFrame whose column names are to be normalized.
    
    Returns:
    pd.DataFrame: DataFrame with normalized column names.
    """
    df.columns = df.columns.str.strip()  # Remove leading/trailing whitespace
    df.columns = df.columns.str.lower()  # Convert to lowercase
    df.columns = df.columns.str.replace(r'\s+', '_', regex=True)  # Replace spaces with underscores
    df.columns = df.columns.str.replace(r'[^\w\s]', '', regex=True)  # Remove special characters
    
    return df

In [5]:
def save_histograms(dataframe, save_path):

    start_time = time.time()
    
    numeric_columns = dataframe.select_dtypes(include=['number'])
    
    # Create the directory if it doesn't exist
    os.makedirs(save_path, exist_ok=True)
    
    for column in numeric_columns.columns:
        plt.figure(figsize=(8, 6))
        
        sns.histplot(dataframe[column], kde=True, color='blue', bins=20)
        plt.title(f'Histogram with KDE of {column}')
        plt.xlabel(column)
        plt.ylabel('Density')
        
        # Calculate the number of events (N) and format it with thousands separators
        num_events = len(dataframe[column])
        formatted_num_events = '{:,}'.format(num_events)
        
        # Add annotation for N in the top-middle part of the histogram
        plt.text(0.5, 0.95, f'N={formatted_num_events}', fontsize=12, ha='center', transform=plt.gca().transAxes, bbox=dict(facecolor='white', alpha=0.7))
        
        # Save the histogram as a .jpg file with the column name
        file_name = f'{column}.jpg'
        file_path = os.path.join(save_path, file_name)
        plt.savefig(file_path)
        
        print(f'Saved: {file_name}')
        
        plt.close()  # Close the plot to avoid displaying in the notebook

        # Print time elapsed for each whole minute
        elapsed_time = time.time() - start_time
        if int(elapsed_time) % 60 == 0 or num_events == len(dataframe[column]) - 1:
            print(f'Time elapsed: {int(elapsed_time / 60)} minutes')

In [6]:
def execute_query(db_path, query):
    try:
        conn = sqlite3.connect(db_path)
        df = pd.read_sql_query(query, conn)
        return df
        
    except sqlite3.Error as e:
        print(f"An error occurred: {e}")
        return None
        
    finally:
        if conn:
            conn.close()

In [7]:
# Example usage to list all tables
db_path = f"{os.getcwd()}/tmp/cdeschools.sqlite"
query = "SELECT name FROM sqlite_master WHERE type='table';"
df_tables = execute_query(db_path, query)

if df_tables is not None:
    print("Tables in the database:")
    print(df_tables)


Tables in the database:
        name
0    schools
1       frpm
2  satscores


### SCHOOLS

In [61]:
query = """
SELECT
  *
FROM
  schools
"""
df_schools = execute_query(db_path, query)

In [62]:
# df_schools = normalize_column_names(df_schools)

In [63]:
df_schools.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17686 entries, 0 to 17685
Data columns (total 49 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   CDSCode      17686 non-null  object 
 1   NCESDist     16656 non-null  object 
 2   NCESSchool   12646 non-null  object 
 3   StatusType   17686 non-null  object 
 4   County       17686 non-null  object 
 5   District     17686 non-null  object 
 6   School       16317 non-null  object 
 7   Street       17392 non-null  object 
 8   StreetAbr    17392 non-null  object 
 9   City         17393 non-null  object 
 10  Zip          17393 non-null  object 
 11  State        17393 non-null  object 
 12  MailStreet   17394 non-null  object 
 13  MailStrAbr   17394 non-null  object 
 14  MailCity     17394 non-null  object 
 15  MailZip      17394 non-null  object 
 16  MailState    17394 non-null  object 
 17  Phone        11717 non-null  object 
 18  Ext          540 non-null    object 
 19  Webs

In [60]:
df_schools.head(3)

,CDSCode,NCESDist,NCESSchool,StatusType,County,District,School,Street,StreetAbr,City,Zip,State,MailStreet,MailStrAbr,MailCity,MailZip,MailState,Phone,Ext,Website,OpenDate,ClosedDate,Charter,CharterNum,FundingType,DOC,DOCType,SOC,SOCType,EdOpsCode,EdOpsName,EILCode,EILName,GSoffered,GSserved,Virtual,Magnet,Latitude,Longitude,AdmFName1,AdmLName1,AdmEmail1,AdmFName2,AdmLName2,AdmEmail2,AdmFName3,AdmLName3,AdmEmail3,LastUpdate
0,01100170000000,0691051,None,Active,Alameda,Alameda County Office of Education,None,313 West Winton Avenue,313 West Winton Ave.,Hayward,94544-1136,CA,313 West Winton Avenue,313 West Winton Ave.,Hayward,94544-1136,CA,(510) 887-0152,None,www.acoe.org,None,None,NaN,None,None,00,County Office of Education (COE),None,None,None,None,None,None,None,None,None,NaN,37.658212,-122.09713,L Karen,Monroe,lkmonroe@acoe.org,None,None,None,None,None,None,2015-06-23
1,01100170109835,0691051,10546,Closed,Alameda,Alameda County Office of Education,FAME Public Charter,"39899 Balentine Drive, Suite 335","39899 Balentine Dr., Ste. 335",Newark,94560-5359,CA,"39899 Balentine Drive, Suite 335","39899 Balentine Dr., Ste. 335",Newark,94560-5359,CA,None,None,None,2005-08-29,2015-07-31,1.0,0728,Directly funded,00,County Office of Education (COE),65,K-12 Schools (Public),TRAD,Traditional,ELEMHIGH,Elementary-High Combination,K-12,K-12,P,0.0,37.521436,-121.99391,None,None,None,None,None,None,None,None,None,2015-09-01
2,01100170112607,0691051,10947,Active,Alameda,Alameda County Office of Education,Envision Academy for Arts & Technology,1515 Webster Street,1515 Webster St.,Oakland,94612-3355,CA,1515 Webster Street,1515 Webster St.,Oakland,94612,CA,(510) 596-8901,None,www.envisionacademy.org/,2006-08-28,None,1.0,0811,Directly funded,00,County Office of Education (COE),66,High Schools (Public),TRAD,Traditional,HS,High School,9-12,9-12,N,0.0,37.804520,-122.26815,Laura,Robell,laura@envisionacademy.org,None,None,None,None,None,None,2015-06-18


In [55]:
df_schools.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
CDSCode,17686,17686,01100170000000,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NCESDist,16656,1193,0622710,1342,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NCESSchool,12646,12321,12271,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
StatusType,17686,4,Active,11708,NaN,NaN,NaN,NaN,NaN,NaN,NaN
County,17686,58,Los Angeles,3636,NaN,NaN,NaN,NaN,NaN,NaN,NaN
District,17686,1411,Los Angeles Unified,1342,NaN,NaN,NaN,NaN,NaN,NaN,NaN
School,16317,13875,Lincoln Elementary,48,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Street,17392,13593,1398 Sperber Road,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN
StreetAbr,17392,13633,1398 Sperber Rd.,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN
City,17393,1165,Los Angeles,837,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
save_histograms(df_schools, f"{save_path}/df_schools")

Saved: Charter.jpg
Time elapsed: 0 minutes
Saved: Magnet.jpg
Time elapsed: 0 minutes
Saved: Latitude.jpg
Time elapsed: 0 minutes
Saved: Longitude.jpg
Time elapsed: 0 minutes


In [57]:
number = 10

for column in df_schools.columns:
    print("Column name:", column)
    print("Data type:", df_schools[column].dtype)
    print(f"First {number} unique values:", df_schools[column].unique()[:number])
    print("Count of unique values:", df_schools[column].nunique())
    print("Count of NULL values:", df_schools[column].isnull().sum(), "\n")

Column name: CDSCode
Data type: object
First 10 unique values: ['01100170000000' '01100170109835' '01100170112607' '01100170118489'
 '01100170123968' '01100170124172' '01100170125567' '01100170130302'
 '01100170130401' '01100170130419']
Count of unique values: 17686
Count of NULL values: 0 

Column name: NCESDist
Data type: object
First 10 unique values: ['0691051' '0600002' '0600003' None '0601397' '0601770' '0601860'
 '0604740' '0607800' '0612630']
Count of unique values: 1193
Count of NULL values: 1030 

Column name: NCESSchool
Data type: object
First 10 unique values: [None '10546' '10947' '12283' '12844' '12901' '13008' '09264' '06830'
 '09265']
Count of unique values: 12321
Count of NULL values: 5040 

Column name: StatusType
Data type: object
First 10 unique values: ['Active' 'Closed' 'Merged' 'Pending']
Count of unique values: 4
Count of NULL values: 0 

Column name: County
Data type: object
First 10 unique values: ['Alameda' 'Alpine' 'Amador' 'Butte' 'Calaveras' 'Colusa' 'Cont

### FRPM - Free or Reduced-Price meals

In [35]:
query = """
SELECT
  *
FROM
  frpm
"""
df_frpm = execute_query(db_path, query)

In [36]:
# df_frpm = normalize_column_names(df_frpm)

In [37]:
df_frpm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10395 entries, 0 to 10394
Data columns (total 28 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   Academic Year                                10393 non-null  object 
 1   County Code                                  10393 non-null  object 
 2   District Code                                10393 non-null  float64
 3   School Code                                  10393 non-null  object 
 4   County Name                                  10393 non-null  object 
 5   District Name                                10393 non-null  object 
 6   School Name                                  10393 non-null  object 
 7   District Type                                10393 non-null  object 
 8   School Type                                  9989 non-null   object 
 9   Educational Option Type                      9989 non-null   object 
 10

In [59]:
df_frpm.head(3)

,Academic Year,County Code,District Code,School Code,County Name,District Name,School Name,District Type,School Type,Educational Option Type,NSLP Provision Status,Charter School (Y/N),Charter School Number,Charter Funding Type,IRC,Low Grade,High Grade,Enrollment (K-12),Free Meal Count (K-12),Percent (%) Eligible Free (K-12),FRPM Count (K-12),Percent (%) Eligible FRPM (K-12),Enrollment (Ages 5-17),Free Meal Count (Ages 5-17),Percent (%) Eligible Free (Ages 5-17),FRPM Count (Ages 5-17),Percent (%) Eligible FRPM (Ages 5-17),2013-14 CALPADS Fall 1 Certification Status
0,2014-2015,01,10017.0,0109835,Alameda,Alameda County Office of Education,FAME Public Charter,County Office of Education (COE),K-12 Schools (Public),Traditional,None,1.0,0728,Directly funded,1.0,K,12,1087.0,565.0,0.519779,715.0,0.657774,1070.0,553.0,0.516822,702.0,0.656075,1.0
1,2014-2015,01,10017.0,0112607,Alameda,Alameda County Office of Education,Envision Academy for Arts & Technology,County Office of Education (COE),High Schools (Public),Traditional,None,1.0,0811,Directly funded,1.0,9,12,395.0,186.0,0.470886,186.0,0.470886,376.0,182.0,0.484043,182.0,0.484043,1.0
2,2014-2015,01,10017.0,0118489,Alameda,Alameda County Office of Education,Aspire California College Preparatory Academy,County Office of Education (COE),High Schools (Public),Traditional,None,1.0,1049,Directly funded,1.0,9,12,244.0,134.0,0.549180,175.0,0.717213,230.0,128.0,0.556522,168.0,0.730435,1.0


In [39]:
df_frpm.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Academic Year,10393,1,2014-2015,10393,NaN,NaN,NaN,NaN,NaN,NaN,NaN
County Code,10393,58,19,2283,NaN,NaN,NaN,NaN,NaN,NaN,NaN
District Code,10393.0,NaN,NaN,NaN,65648.162802,11101.123192,10017.0,64733.0,67082.0,69625.0,76901.0
School Code,10393,9991,0000001,359,NaN,NaN,NaN,NaN,NaN,NaN,NaN
County Name,10393,58,Los Angeles,2283,NaN,NaN,NaN,NaN,NaN,NaN,NaN
District Name,10393,1010,Los Angeles Unified,997,NaN,NaN,NaN,NaN,NaN,NaN,NaN
School Name,10393,8693,"Nonpublic, Nonsectarian Schools",359,NaN,NaN,NaN,NaN,NaN,NaN,NaN
District Type,10393,8,Unified School District,6921,NaN,NaN,NaN,NaN,NaN,NaN,NaN
School Type,9989,17,Elementary Schools (Public),5616,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Educational Option Type,9989,12,Traditional,8740,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
# save_histograms(df_frpm, f"{save_path}/df_frpm")

In [42]:
for column in df_frpm.columns:
    print("Column name:", column)
    print("Data type:", df_frpm[column].dtype)
    print("First five unique values:", df_frpm[column].unique()[:5])
    print("Count of unique values:", df_frpm[column].nunique())
    print("Count of NULL values:", df_frpm[column].isnull().sum(), "\n")

Column name: Academic Year
Data type: object
First five unique values: ['2014-2015' None]
Count of unique values: 1
Count of NULL values: 2 

Column name: County Code
Data type: object
First five unique values: ['01' '02' '03' '04' '05']
Count of unique values: 58
Count of NULL values: 2 

Column name: District Code
Data type: float64
First five unique values: [10017. 31609. 31617. 61119. 61127.]
Count of unique values: 1022
Count of NULL values: 2 

Column name: School Code
Data type: object
First five unique values: ['0109835' '0112607' '0118489' '0123968' '0124172']
Count of unique values: 9991
Count of NULL values: 2 

Column name: County Name
Data type: object
First five unique values: ['Alameda' 'Alpine' 'Amador' 'Butte' 'Calaveras']
Count of unique values: 58
Count of NULL values: 2 

Column name: District Name
Data type: object
First five unique values: ['Alameda County Office of Education'
 'California School for the Blind (State Special Schl)'
 'California School for the Deaf

### SATSCORES

cds: Identifier for the institution or dataset 

rtype: Type of report or record 

enroll12: Enrollment number in 12th grade

NumTstTakr: Number of test takers

AvgScrRead: Average reading score

AvgScrMath: Average math score

AvgScrWrite: Average writing score

NumGE1500: Number of students scoring 1500 or higher

PctGE1500: Percentage of students scoring 1500 or higher


In [43]:
query = """
SELECT
  *
FROM
  satscores
"""
df_satscores = execute_query(db_path, query)

In [44]:
# df_satscores = normalize_column_names(df_satscores)

In [45]:
df_satscores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2331 entries, 0 to 2330
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   cds          2331 non-null   object 
 1   rtype        2331 non-null   object 
 2   sname        1753 non-null   object 
 3   dname        2273 non-null   object 
 4   cname        2330 non-null   object 
 5   enroll12     2331 non-null   int64  
 6   NumTstTakr   2331 non-null   int64  
 7   AvgScrRead   1733 non-null   float64
 8   AvgScrMath   1733 non-null   float64
 9   AvgScrWrite  1733 non-null   float64
 10  NumGE1500    1733 non-null   float64
 11  PctGE1500    1733 non-null   float64
dtypes: float64(5), int64(2), object(5)
memory usage: 218.7+ KB


In [46]:
df_satscores.head()

,cds,rtype,sname,dname,cname,enroll12,NumTstTakr,AvgScrRead,AvgScrMath,AvgScrWrite,NumGE1500,PctGE1500
0,00000000000000,X,None,None,None,496901,210706,489.0,500.0,484.0,93334.0,44.30
1,01000000000000,C,None,None,Alameda,16978,8855,516.0,536.0,517.0,4900.0,55.34
2,01100170000000,D,None,Alameda County Office of Education,Alameda,398,88,418.0,418.0,417.0,14.0,15.91
3,01100170109835,S,FAME Public Charter,Alameda County Office of Education,Alameda,62,17,503.0,546.0,505.0,9.0,52.94
4,01100170112607,S,Envision Academy for Arts & Technology,Alameda County Office of Education,Alameda,75,71,397.0,387.0,395.0,5.0,7.04


In [47]:
df_satscores.tail()

,cds,rtype,sname,dname,cname,enroll12,NumTstTakr,AvgScrRead,AvgScrMath,AvgScrWrite,NumGE1500,PctGE1500
2326,58727365830054,S,Lincoln (Abraham) (Alternative),Marysville Joint Unified,Yuba,97,0,NaN,NaN,NaN,NaN,NaN
2327,58727365830138,S,Marysville Charter Academy for the Arts,Marysville Joint Unified,Yuba,41,29,501.0,494.0,484.0,16.0,55.17
2328,58727365835202,S,Marysville High,Marysville Joint Unified,Yuba,197,53,489.0,513.0,487.0,24.0,45.28
2329,58727690000000,D,None,Wheatland Union High,Yuba,160,54,480.0,475.0,463.0,21.0,38.89
2330,58727695838305,S,Wheatland Union High,Wheatland Union High,Yuba,160,54,480.0,475.0,463.0,21.0,38.89


In [48]:
df_satscores.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
cds,2331,2331,00000000000000,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
rtype,2331,4,S,1753,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sname,1753,1669,Middle College High,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
dname,2273,520,Los Angeles Unified,202,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cname,2330,57,Los Angeles,469,NaN,NaN,NaN,NaN,NaN,NaN,NaN
enroll12,2331.0,NaN,NaN,NaN,834.767911,10808.227243,0.0,71.0,229.0,500.0,496901.0
NumTstTakr,2331.0,NaN,NaN,NaN,360.759331,4632.416926,0.0,10.0,86.0,231.5,210706.0
AvgScrRead,1733.0,NaN,NaN,NaN,480.242354,57.168344,308.0,440.0,482.0,520.0,653.0
AvgScrMath,1733.0,NaN,NaN,NaN,484.99019,61.900589,289.0,445.0,484.0,525.0,699.0
AvgScrWrite,1733.0,NaN,NaN,NaN,472.955568,55.200661,312.0,435.0,472.0,509.0,671.0


In [49]:
save_histograms(df_satscores, f"{save_path}/df_satscores")

Saved: enroll12.jpg
Time elapsed: 0 minutes
Saved: NumTstTakr.jpg
Time elapsed: 0 minutes
Saved: AvgScrRead.jpg
Time elapsed: 0 minutes
Saved: AvgScrMath.jpg
Time elapsed: 0 minutes
Saved: AvgScrWrite.jpg
Saved: NumGE1500.jpg
Saved: PctGE1500.jpg


In [50]:
for column in df_satscores.columns:
    print("Column name:", column)
    print("Data type:", df_satscores[column].dtype)
    print("First five unique values:", df_satscores[column].unique()[:5])
    print("Count of unique values:", df_satscores[column].nunique())
    print("Count of NULL values:", df_satscores[column].isnull().sum(), "\n")

Column name: cds
Data type: object
First five unique values: ['00000000000000' '01000000000000' '01100170000000' '01100170109835'
 '01100170112607']
Count of unique values: 2331
Count of NULL values: 0 

Column name: rtype
Data type: object
First five unique values: ['X' 'C' 'D' 'S']
Count of unique values: 4
Count of NULL values: 0 

Column name: sname
Data type: object
First five unique values: [None 'FAME Public Charter' 'Envision Academy for Arts & Technology'
 'Aspire California College Preparatory Academy'
 'Alameda Science and Technology Institute']
Count of unique values: 1669
Count of NULL values: 578 

Column name: dname
Data type: object
First five unique values: [None 'Alameda County Office of Education' 'Alameda Unified'
 'Albany City Unified' 'Berkeley Unified']
Count of unique values: 520
Count of NULL values: 58 

Column name: cname
Data type: object
First five unique values: [None 'Alameda' 'Amador' 'Butte' 'Calaveras']
Count of unique values: 57
Count of NULL values: 

#### Business Case

FRPM – Student Poverty Data Case Study

The California Department of Education (CDE) oversees the state's public school system, which is responsible for the education of more than six million children and young adults in more than 10,000 schools with 295,000 teachers. 

The Director of Education sent you an email earlier today:

From: Director of Education 
To: You (Analytics Engineer)
Subject: Impact of free school meals and school type on education

Hi - we’re glad to have assigned you to this project. I will soon be discussing the impact of free school meals and school type on exam scores to the Board of Education. I need answers to a few questions so I can bring the relevant data and insights to the meeting. Can you help me? 

Here’s how you can get the data: 
●	Use this link to fetch the database http://2016.padjo.org/files/data/starterpack/cde-schools/cdeschools.sqlite  
●	It’s in SQLite format so it should be very familiar for you 
●	You can use any tool that can help you navigate the data 
 
I need to know: 
●	What are the 10 best high schools for maths scores?
●	Which counties don’t have any SAT scores?
●	Are locally funded charter schools more likely than directly funded charter schools to enrol students on the FRPM (Free or Reduced-Price meals) program?
●	Does being on the FRPM program impact math scores?
●	Which school types are the best at achieving good reading and writing scores for students on the FRPM program?

We don’t have a lot of time, so I want you to pull the data together and run a quick analysis on each of these and report back tomorrow. Some of the queries are open to interpretation and I’d like for you to send me your answers and opinions to the questions above and let me know your reasoning. 

I’d like a clear (and brief) word document, rather than a powerpoint, accompanied by any data visualisations you consider necessary. 

Thanks, and we look forward to hearing from you.
-	Director of Education


#### GOAL 1: impact of free school meals on exam scores

#### GOAL 2: impact of school type on exam scores 

●	Q1: What are the 10 best high schools for maths scores?

●	Q2: Which counties don’t have any SAT scores?

●	Q3: Are locally funded charter schools more likely than directly funded charter schools to enrol students on the FRPM (Free or Reduced-Price meals) program?

●	Q4: Does being on the FRPM program impact math scores?

●	Q5: Which school types are the best at achieving good reading and writing scores for students on the FRPM program?

Some of the queries are open to interpretation

I’d like for you to send me your answers and opinions to the questions above and

let me know your reasoning. 